# TrAdaBoost Transfer Learning ULSA -> UENG
# Model 1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from scipy.stats import kstest
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

In [2]:
from transformers.features_missing_vals import imputation_pipeline
from transformers.ethnic_groups import CombineEthnicGroups

In [3]:
from transfer_learning.TrAdaBoost import TrAdaBoost

In [ ]:
from model_evaluation.cross_val import direct_transfer_cv, TrAdaBoost_transfer_cv

# Params

In [5]:
source_school = 'ULSA'
target_school = 'UENG'

In [6]:
# Model features
cols = ['STDNT_FEMALE',
        'HS_GPA_BIN',
        'CURR_GPA',
        'HS_CALC_IND',
        'SNGL_PRNT_IND',
        'STDNT_ASIAN_IND',
        'STDNT_BLACK_IND',
        'STDNT_HSPNC_IND',
        'STDNT_NTV_AMRCN_HWIAN_IND',
        'STDNT_MULTI_ETHNC_IND',
        'STDNT_ETHNC_NAN',
        'SAT_ACT_TOTAL_BIN',
        'ACAD_LOAD',
        'No_grades_at_all',
        'Grade_Overall_I_for_1_and_more_courses',
        'Grade_W_for_1_course',
        'Grade_W_for_2_courses',
        'Grade_W_for_3_and_more_courses',
        'Grade_NR_for_1_and_more_courses']

In [7]:
random_state = 1

In [8]:
clf = LogisticRegression(penalty=None, random_state=random_state, max_iter=10000)

In [9]:
# Parameters for fairness metrics

# Priviledged values of attributes
attribites_dic = {
    'STDNT_ETHNC_grouped': 0, # white
    'STDNT_FEMALE': 0 # male
    }

# The threshold for intervention is based on the percentile of predicted dropout probabilities, 
# specifically targeting the top 10% of students with the highest dropout probabilities
treshold=[10]

# Percentile values of predicted dropout probabilities that determine the risk ranking of a student
rank_thresholds = [90, 95, 97, 99, 99.5]

# Data

In [10]:
features_df = pd.read_csv('features_df.csv', low_memory=False)
source_df = features_df[features_df['PRMRY_CRER_CD']==source_school].copy()
target_df = features_df[features_df['PRMRY_CRER_CD']==target_school].copy()

# Reset target index to ensure that cross-validation indices will be accurately applied
target_df.reset_index(drop=True, inplace=True)

In [ ]:
# Group together all ethnic groups, excluding white and asian populations, as they lack a sufficient 
# number of dropouts for a meaningful fairness analysis

# Add new column 'STDNT_ETHNC_grouped' with the following values: 0 for white, 1 for asian, and 2 for other
combine_transformer = CombineEthnicGroups()
target_df = combine_transformer.fit_transform(target_df)

target_df.groupby('STDNT_ETHNC_grouped')[['y_ENROLLED_1_YEAR_LATER']].agg(['count', 'sum'])

In [12]:
# Impute missing values and apply WoE transformation for the direct transfer
source_df = combine_transformer.fit_transform(source_df)
source_imp = imputation_pipeline.fit_transform(source_df, source_df['y_ENROLLED_1_YEAR_LATER'])
target_imp = imputation_pipeline.transform(target_df)

# Train and test data sets in cross-validation
The target dataset will be partitioned into a training set comprising 33.3% of the data and a testing set comprising 66.6%.
For TrAdaBoost, the training set will encompass the source dataset and 33.3% of the target dataset, while the testing set for TrAdaBoost will consist of the remaining 66.6% from the target dataset. This specific partition is chosen because the testing set needs to contain a sufficient number of dropouts for the evaluation of AUC and other metrics. In cases with a low dropout count, the confidence interval of AUC tends to be broad, making the AUC estimate less reliable. Specifically, 66.6% of the target sample includes 84-85 instances of student dropouts.

In [13]:
stratified_split = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)
splits = stratified_split.split(target_df, target_df['y_ENROLLED_1_YEAR_LATER'])

In [14]:
# Generate indices for the training and testing datasets for every split in the cross-validation
train_target_indx = []
test_target_indx = []

for i, (test_index, train_index) in enumerate(splits):
    train_target_indx.append(train_index)
    test_target_indx.append(test_index)

In [ ]:
# Check the cross-validation splits
for i in range(stratified_split.get_n_splits()):
    train_size = len(train_target_indx[i])
    test_size = len(test_target_indx[i])
    train_dropout = target_df.loc[train_target_indx[i], 'y_ENROLLED_1_YEAR_LATER'].mean()
    test_dropout = target_df.loc[test_target_indx[i], 'y_ENROLLED_1_YEAR_LATER'].mean()
    test_dropout_num = target_df.loc[test_target_indx[i], 'y_ENROLLED_1_YEAR_LATER'].sum()
    print(f"Split {i}: Training set size={train_size}, testing set size={test_size}")
    print(f"Split {i}: Training dropout rate={train_dropout:.2%}, testing dropout rate={test_dropout:.2%},", 
          f"testing number of dropouts={test_dropout_num:.0f}")

# Direct transfer
To ensure comparability between direct transfer and TrAdaBoost, the evaluation of direct transfer will be conducted using identical cross-validation test sets as for TrAdaBoost. Using different samples to compare these methods may result in variations in model evaluation metrics, which could be attributed to sample differences rather than indicating the superiority of one model over the other.

## Source model

In [16]:
# Fit the source model
clf_source = clone(clf)
clf_source.fit(source_imp[cols], source_imp['y_ENROLLED_1_YEAR_LATER'])

LogisticRegression(max_iter=10000, penalty=None, random_state=1)

## Cross-validating the direct transfer of the source model
The performance of the source model 'clf_source' trained on the source dataset (refer to the cell above), is evaluated on three  target test sets, specified by the indices 'test_target_indx'. The evaluation includes classification accuracy and fairness metrics.

In [17]:
cv_res_direct = direct_transfer_cv(
    target_imp, # Target data set with imputed missing values
    test_target_indx, # Indices of the testing sets for each cross-validation splits
    clf_source, cols, # Fitted source model and its features
    'Direct transfer', 
    attribites_dic, treshold, rank_thresholds) # Fairness metrics' parameters 

## Classification accuracy

### AUC

In [18]:
cv_res_direct['AUC']

,split_0,split_1,split_2,mean,se
Direct transfer,0.779717,0.801368,0.761819,0.780968,0.011434


### Pietra Index and Kolmogorov-Smirnov test

In [19]:
# Pietra index
cv_res_direct['pietra_index']

,split_0,split_1,split_2,mean,se
Direct transfer,0.473917,0.521246,0.456717,0.48396,0.019293


In [20]:
# KS p-value
cv_res_direct['KS_pvalue'].iloc[:, :3]

,split_0,split_1,split_2
Direct transfer,8.208334e-18,1.790709e-21,1.648702e-16


## Fairness metrics

### Sliced AUCs

In [21]:
cv_res_direct['sliced_AUCs']

,Attribute,Value,split_0,split_1,split_2,mean,se
0,STDNT_ETHNC_grouped,1,0.712773,0.746055,0.691876,0.716901,0.015776
1,STDNT_ETHNC_grouped,0,0.848525,0.843866,0.823676,0.838689,0.007626
2,STDNT_ETHNC_grouped,2,0.776948,0.83042,0.773491,0.793620,0.018427
3,STDNT_FEMALE,0,0.777641,0.784484,0.767998,0.776708,0.004782
4,STDNT_FEMALE,1,0.786747,0.865679,0.747599,0.800008,0.034726


### Equal opportunity difference

In [22]:
cv_res_direct['equal_opportunity_diff']

,Attribute,Value,split_0,split_1,split_2,mean,se
0,STDNT_ETHNC_grouped,1,-0.344156,-0.259756,-0.361039,-0.321650,0.031329
1,STDNT_ETHNC_grouped,0,0,0,0,0.000000,0.000000
2,STDNT_ETHNC_grouped,2,0.095238,0.129374,0.135714,0.120109,0.012569
3,STDNT_FEMALE,0,0,0,0,0.000000,0.000000
4,STDNT_FEMALE,1,0.057692,0.006146,0.039435,0.034424,0.015090


### Generalized entropy index

In [23]:
cv_res_direct['gen_entropy']

,split_0,split_1,split_2,mean,se
generalized_entropy,0.488372,0.357143,0.511905,0.452473,0.048147
between_groups,0.055878,0.029314,0.076651,0.053948,0.013699


### Generalized entropy index that takes into account the risk ranks of students

In [24]:
cv_res_direct['gen_entropy_ranks']

,split_0,split_1,split_2,mean,se
generalized_entropy_ranks,0.624671,0.494955,0.640999,0.586875,0.046201
between_groups_ranks,0.041708,0.031849,0.074986,0.049514,0.013050


# TrAdaBoost

In [25]:
TrAdaBoost_model = TrAdaBoost(estimator=clone(clf), max_num_iterations=10)

## Cross-validating the TrAdaBoost
In every cross-validation split, the TrAdaBoost model is trained using the source dataset combined with 33.3% of the target dataset. Subsequently, the model is evaluated on the remaining 66.6% of the target dataset. The evaluation includes classification accuracy and fairness metrics.

In [26]:
cv_res_TrAdaBoost = TrAdaBoost_transfer_cv(
    source_df, target_df, # Source and target data sets
    train_target_indx, # Indices of the training sets for cross-validation splits
    test_target_indx, # Indices of the testing sets for cross-validation splits
    imputation_pipeline, # Pipeline to impute missing values and apply WoE transformation
    TrAdaBoost_model, 
    cols, # Model features
    'TrAdaBoost', 
    attribites_dic, treshold, rank_thresholds) # Fairness metrics' parameters 

## Classification accuracy

### AUC

In [27]:
pd.concat([cv_res_TrAdaBoost['AUC'], cv_res_direct['AUC']])

,split_0,split_1,split_2,mean,se
TrAdaBoost,0.78956,0.829711,0.758234,0.792502,0.020686
Direct transfer,0.779717,0.801368,0.761819,0.780968,0.011434


### Pietra index and Kolmogorov-Smirnov test

In [28]:
# Pietra index
pd.concat([cv_res_TrAdaBoost['pietra_index'], cv_res_direct['pietra_index']])

,split_0,split_1,split_2,mean,se
TrAdaBoost,0.477261,0.540177,0.459435,0.492291,0.024490
Direct transfer,0.473917,0.521246,0.456717,0.483960,0.019293


In [29]:
# KS p-value
pd.concat([cv_res_TrAdaBoost['KS_pvalue'].iloc[:,:3], 
           cv_res_direct['KS_pvalue'].iloc[:,:3]])

,split_0,split_1,split_2
TrAdaBoost,4.503118e-18,3.723825e-23,1.036112e-16
Direct transfer,8.208334e-18,1.790709e-21,1.648702e-16


## Fairness metrics

### Sliced AUCs

In [30]:
cv_res_TrAdaBoost['sliced_AUCs']

,Attribute,Value,split_0,split_1,split_2,mean,se
0,STDNT_ETHNC_grouped,1,0.699312,0.815809,0.67244,0.729187,0.044000
1,STDNT_ETHNC_grouped,0,0.860566,0.853952,0.823117,0.845878,0.011540
2,STDNT_ETHNC_grouped,2,0.786622,0.817248,0.775329,0.793066,0.012522
3,STDNT_FEMALE,0,0.779221,0.812963,0.754916,0.782366,0.016830
4,STDNT_FEMALE,1,0.795702,0.853142,0.732621,0.793822,0.034804


In [31]:
cv_res_TrAdaBoost['sliced_AUCs'][['Attribute', 'Value', 'mean', 'se']].merge(
    cv_res_direct['sliced_AUCs'][['Attribute', 'Value', 'mean', 'se']], 
    on=['Attribute', 'Value'],
    suffixes=('_TrAdaBoost', '_direct')
    )[['Attribute', 'Value', 'mean_TrAdaBoost', 'mean_direct', 'se_TrAdaBoost', 'se_direct']]

,Attribute,Value,mean_TrAdaBoost,mean_direct,se_TrAdaBoost,se_direct
0,STDNT_ETHNC_grouped,1,0.729187,0.716901,0.044000,0.015776
1,STDNT_ETHNC_grouped,0,0.845878,0.838689,0.011540,0.007626
2,STDNT_ETHNC_grouped,2,0.793066,0.793620,0.012522,0.018427
3,STDNT_FEMALE,0,0.782366,0.776708,0.016830,0.004782
4,STDNT_FEMALE,1,0.793822,0.800008,0.034804,0.034726


### Equal opportunity difference

In [32]:
cv_res_TrAdaBoost['equal_opportunity_diff']

,Attribute,Value,split_0,split_1,split_2,mean,se
0,STDNT_ETHNC_grouped,1,-0.138528,-0.159756,-0.253247,-0.183844,0.035238
1,STDNT_ETHNC_grouped,0,0,0,0,0.000000,0.000000
2,STDNT_ETHNC_grouped,2,0.119048,0.129374,0.107143,0.118522,0.006423
3,STDNT_FEMALE,0,0,0,0,0.000000,0.000000
4,STDNT_FEMALE,1,0.011538,-0.023705,-0.023065,-0.011744,0.011643


In [33]:
cv_res_TrAdaBoost['equal_opportunity_diff'][['Attribute', 'Value', 'mean', 'se']].merge(
    cv_res_direct['equal_opportunity_diff'][['Attribute', 'Value', 'mean', 'se']], 
    on=['Attribute', 'Value'],
    suffixes=('_TrAdaBoost', '_direct')
    )[['Attribute', 'Value', 'mean_TrAdaBoost', 'mean_direct', 'se_TrAdaBoost', 'se_direct']]

,Attribute,Value,mean_TrAdaBoost,mean_direct,se_TrAdaBoost,se_direct
0,STDNT_ETHNC_grouped,1,-0.183844,-0.321650,0.035238,0.031329
1,STDNT_ETHNC_grouped,0,0.000000,0.000000,0.000000,0.000000
2,STDNT_ETHNC_grouped,2,0.118522,0.120109,0.006423,0.012569
3,STDNT_FEMALE,0,0.000000,0.000000,0.000000,0.000000
4,STDNT_FEMALE,1,-0.011744,0.034424,0.011643,0.015090


### Generalized entropy index

In [34]:
cv_res_TrAdaBoost['gen_entropy']

,split_0,split_1,split_2,mean,se
generalized_entropy,0.423913,0.323529,0.423913,0.390452,0.033461
between_groups,0.014386,0.014449,0.033232,0.020689,0.006272


In [35]:
cv_res_TrAdaBoost['gen_entropy'][['mean', 'se']].merge(
    cv_res_direct['gen_entropy'][['mean', 'se']], 
    left_index=True, right_index=True,
    suffixes=('_TrAdaBoost', '_direct')
    )[['mean_TrAdaBoost', 'mean_direct', 'se_TrAdaBoost', 'se_direct']]

,mean_TrAdaBoost,mean_direct,se_TrAdaBoost,se_direct
generalized_entropy,0.390452,0.452473,0.033461,0.048147
between_groups,0.020689,0.053948,0.006272,0.013699


### Generalized entropy index that takes into account the risk ranks of students

In [36]:
cv_res_TrAdaBoost['gen_entropy_ranks']

,split_0,split_1,split_2,mean,se
generalized_entropy_ranks,0.646905,0.466857,0.608984,0.574249,0.054800
between_groups_ranks,0.026627,0.023209,0.047986,0.032607,0.007753


In [37]:
cv_res_TrAdaBoost['gen_entropy_ranks'][['mean', 'se']].merge(
    cv_res_direct['gen_entropy_ranks'][['mean', 'se']], 
    left_index=True, right_index=True,
    suffixes=('_TrAdaBoost', '_direct')
    )[['mean_TrAdaBoost', 'mean_direct', 'se_TrAdaBoost', 'se_direct']]

,mean_TrAdaBoost,mean_direct,se_TrAdaBoost,se_direct
generalized_entropy_ranks,0.574249,0.586875,0.054800,0.046201
between_groups_ranks,0.032607,0.049514,0.007753,0.013050
